# MindBridge Task Assignment Prototype

This notebook demonstrates how to automate the assignment of review tasks in MindBridge using the MindBridge API. It covers:

- Connecting to a MindBridge analysis and exporting filtered data.
- Assigning tasks to specific users based on data attributes.

Use this notebook as a template for integrating MindBridge task assignment into your audit workflows.

#### Installing the MindBridge API Python Client

This must be the first code cell executed in the notebook to ensure the [MindBridge API Python Client](https://pypi.org/project/mindbridge-api-python-client/) is available for use.

In [0]:
%pip install --upgrade mindbridge-api-python-client
dbutils.library.restartPython()

#### Configuration

Set up the configuration parameters for connecting to the MindBridge analysis and defining the task assignment logic. Here are the variables available to be configured:

- `analysis_url`: The URL of the MindBridge analysis from which to export data. In a later cell this is parsed to extract the analysis result ID and the MindBridge server URL. In many cases you may want to run this immediately after and analysis run, so you may alternatively determine these as part of that process.
- `data_table_query`: A dictionary defining the query to filter the data table rows for which tasks will be created.
- `data_table_logical_name`: The logical name of the data table to export data from (e.g., "gl_journal_lines").
- `column_for_assigning`: The column name in the data table used to determine task assignment.
- `assigned_user_map`: A mapping of `column_for_assigning` values to MindBridge user email addresses for task assignment.
- `MINDBRIDGE_API_TOKEN` secret scope must be created in Databricks containing a valid MindBridge API token. For more details see the first notebook in this repository.

In [0]:
from datetime import date

analysis_url = "https://psus.mindbridge.ai/app/organization/68f7dc86efccc75265245b0e/engagement/68f7dc87efccc75265245b10/analysis/68f7dc9eefccc75265245bf4/analyze/financial-statements?productCode=GENERAL_LEDGER#facetBarState=JTdCJTIydmlzaWJsZV9mYWNldF9pZHMlMjIlM0ElNUIlNUQlMkMlMjJmYWNldF9zdGF0ZXMlMjIlM0ElNUIlNUQlN0Q="
data_table_query = {
    "effective_date": {"$gte": date(2022, 4, 1), "$lt": date(2022, 6, 1)},
    "risk": {"$gte": 15_00},
}
data_table_logical_name = "gl_journal_lines"
column_for_assigning = "company_code"
assigned_user_map = {
    "122644": "kevin.paulson@mindbridge.ai",
    "8477": "michelle.alexander@mindbridge.ai",
}

#### Processing

All cells after this point perform the task assignment process, they wouldn't typically require modification.

In [0]:
from urllib3.util import parse_url

parsed_url = parse_url(analysis_url)
mindbridge_url = parsed_url.host

parsed_url_path = parsed_url.path.split("/")
analysis_result_id = parsed_url_path[parsed_url_path.index("analysis") + 1]

In [0]:
import mindbridgeapi as mbapi

# Load your token from the secret scope
token = dbutils.secrets.get(
    scope="mindbridge-api-tutorials", key="MINDBRIDGE_API_TOKEN"
)

# Create a connection to the server
server = mbapi.Server(url=mindbridge_url, token=token)

# Get the analysis
analysis_result = server.analysis_results.get_by_id(analysis_result_id)
analysis = server.analyses.get_by_id(analysis_result.analysis_id)

print("Available data tables:")
for x in analysis.data_tables:
    print(f"- logical_name: {x.logical_name} (type: {x.type}, id: {x.id})")

server.analyses.restart_data_tables(analysis)
data_table = next(
    x for x in analysis.data_tables if x.logical_name == data_table_logical_name
)
print(
    f"Using logical_name: {data_table.logical_name} (type: {data_table.type}, id: {data_table.id})"
)

assigned_user_map_id = {}
for key, assigned_user_email in assigned_user_map.items():
    user, *others = server.users.get(json={"email": assigned_user_email})
    if not user:
        raise ValueError(f"User with email {assigned_user_email} not found")

    if others:
        raise Exception(
            f"Unexpected error: multiple users found with email {assigned_user_email}"
        )

    assigned_user_map_id[key] = user.id

In [0]:
from pathlib import Path
from tempfile import NamedTemporaryFile
import csv

with NamedTemporaryFile(delete=False) as temp_file:
    temp_file_path = Path(temp_file.name)

print(f"Exporting to: {temp_file_path}")
async_result = server.data_tables.export(
    data_table,
    fields=["rowid", "txid", "risk", "effective_date", column_for_assigning],
    query=data_table_query,
)
server.data_tables.wait_for_export(async_result)
temp_file_path = server.data_tables.download(
    async_result, output_file_path=temp_file_path
)

with temp_file_path.open(newline="", encoding="utf_8") as infile:
    reader = csv.DictReader(infile)
    print("Creating Tasks for:")
    for row in reader:
        row_id = row["rowid"]
        transaction_id = row["txid"]
        print(
            f"- row_id: {row_id}, transaction_id: {transaction_id}, risk: {row['risk']}, effective_date: {row['effective_date']}"
        )
        assigned_id = assigned_user_map_id[row[column_for_assigning]]
        task = mbapi.TaskItem(
            row_id=row_id,
            transaction_id=transaction_id,
            type=mbapi.TaskType.ENTRY,
            status=mbapi.TaskStatus.OPEN,
            engagement_id=analysis.engagement_id,
            analysis_result_id=analysis_result.id,
            audit_areas=["Audit Area 1"],
            assigned_id=assigned_id,
        )
        existing_task = next(
            server.tasks.get(
                json={
                    "analysisId": analysis.id,
                    "analysisResultId": task.analysis_result_id,
                    "transactionId": task.transaction_id,
                    "rowId": task.row_id,
                }
            ),
            None,
        )
        if existing_task:
            print(f"   - Task already exists as {existing_task.id}")
        else:
            task = server.tasks.create(task)
            print(f"    - Created task: {task.id}")

print("Done.")
temp_file_path.unlink()